In [73]:
# x_1 = 7, x_2 = 6, s_3 = 0, s_4 = 0
obj = [5, 3]
consts = [[[3, 2],2400],[[0, 1],800], [[2, 0],1200]]
#basis [s1, s2]

In [74]:
def simplexMethod(objFunction: list, constraints: list, isMax : bool = True) -> list:
    
    """
    Solves the linear programming problem using the Simplex method.

    Parameters:
    @param obj_function (list): The coefficients of the objective function.
    @param constraints (list): A list of constraints, where each constraint is a list containing the coefficients and the right-hand side value.

    Returns:
    list: The solution to the linear programming problem.
    """

    originalNum = len(constraints)
    internalCount = 0
    originalObj = objFunction.copy()
    
    for i in range(originalNum):
        objFunction.append(0)
        for j in range(originalNum):
            if internalCount%(len(constraints)+1) == 0:
                constraints[i][0].append(1)
            else:
                constraints[i][0].append(0)
            print(f'constraints[{i}][0]: ', constraints[i][0])
            internalCount += 1
    result = []
    resultIndex = [-1 for i in range(len(constraints))]
    basis = [0 for i in range(len(constraints))]
    print("basis: ", basis)
    print(len(constraints[0][0]))
    print("objFunction: ", objFunction)
    print(len(objFunction))
    z_j = [sum(basis[j]*constraints[j][0][i] for j in range(len(constraints))) for i in range(len(constraints[0][0]))]
    print("z_j: ", z_j)
    c_j_minus_z_j = [objFunction[i] - z_j[i] for i in range(len(z_j))]
    print("c_j_minus_z_j: ", c_j_minus_z_j)

    while max(c_j_minus_z_j) > 0:  
        maxIndex = c_j_minus_z_j.index(max(c_j_minus_z_j))
        print("maxIndex: ", maxIndex)

        ratios = []
        for i in range(len(constraints)):
            if constraints[i][0][maxIndex] > 0:
                ratios.append((constraints[i][1]/constraints[i][0][maxIndex], i))
        
        if not ratios:
            return "Unbounded solution"
            
        minRatio, minIndex = min(ratios)
        print("minRatio: ", minRatio)
        print("minIndex: ", minIndex)

        basis[minIndex] = objFunction[maxIndex]
        resultIndex[minIndex] = maxIndex

        print(f'Result Index: {resultIndex}')

        pivot = constraints[minIndex][0][maxIndex]

        old_row = constraints[minIndex].copy()

        constraints[minIndex][0] = [x/pivot for x in constraints[minIndex][0]]
        constraints[minIndex][1] = constraints[minIndex][1]/pivot
        print("constraints[minIndex][0]: ", constraints[minIndex][0])
        print("constraints[minIndex][1]: ", constraints[minIndex][1])

        for i in range(len(constraints)):
            if i != minIndex:
                multiplier = constraints[i][0][maxIndex]
                constraints[i][0] = [constraints[i][0][j] - multiplier*constraints[minIndex][0][j] 
                                   for j in range(len(constraints[0][0]))]
                constraints[i][1] = constraints[i][1] - multiplier*constraints[minIndex][1]
        
        print("results: ", [constraints[i][1] for i in range(len(constraints))])
        
        z_j = [sum(basis[j]*constraints[j][0][i] for j in range(len(basis))) for i in range(len(constraints[0][0]))]
        c_j_minus_z_j = [objFunction[i] - z_j[i] for i in range(len(z_j))]

    for i in range(len(originalObj)):
        if i in resultIndex:
            result.append(constraints[resultIndex.index(i)][1])
        else:
            result.append(0)
    objFunction = []
    constraints = []
    return result

print(simplexMethod(obj, consts))

constraints[0][0]:  [3, 2, 1]
constraints[0][0]:  [3, 2, 1, 0]
constraints[0][0]:  [3, 2, 1, 0, 0]
constraints[1][0]:  [0, 1, 0]
constraints[1][0]:  [0, 1, 0, 1]
constraints[1][0]:  [0, 1, 0, 1, 0]
constraints[2][0]:  [2, 0, 0]
constraints[2][0]:  [2, 0, 0, 0]
constraints[2][0]:  [2, 0, 0, 0, 1]
basis:  [0, 0, 0]
5
objFunction:  [5, 3, 0, 0, 0]
5
z_j:  [0, 0, 0, 0, 0]
c_j_minus_z_j:  [5, 3, 0, 0, 0]
maxIndex:  0
minRatio:  600.0
minIndex:  2
Result Index: [-1, -1, 0]
constraints[minIndex][0]:  [1.0, 0.0, 0.0, 0.0, 0.5]
constraints[minIndex][1]:  600.0
results:  [600.0, 800.0, 600.0]
maxIndex:  1
minRatio:  300.0
minIndex:  0
Result Index: [1, -1, 0]
constraints[minIndex][0]:  [0.0, 1.0, 0.5, 0.0, -0.75]
constraints[minIndex][1]:  300.0
results:  [300.0, 500.0, 600.0]
[600.0, 300.0]


In [75]:
profit = [3, 5, 4]
constraintProfit = [[[2, 3, 0],8],[[3, 2, 2], 15],[[0, 4,5], 10]]
print(simplexMethod(profit, constraintProfit))

constraints[0][0]:  [2, 3, 0, 1]
constraints[0][0]:  [2, 3, 0, 1, 0]
constraints[0][0]:  [2, 3, 0, 1, 0, 0]
constraints[1][0]:  [3, 2, 2, 0]
constraints[1][0]:  [3, 2, 2, 0, 1]
constraints[1][0]:  [3, 2, 2, 0, 1, 0]
constraints[2][0]:  [0, 4, 5, 0]
constraints[2][0]:  [0, 4, 5, 0, 0]
constraints[2][0]:  [0, 4, 5, 0, 0, 1]
basis:  [0, 0, 0]
6
objFunction:  [3, 5, 4, 0, 0, 0]
6
z_j:  [0, 0, 0, 0, 0, 0]
c_j_minus_z_j:  [3, 5, 4, 0, 0, 0]
maxIndex:  1
minRatio:  2.5
minIndex:  2
Result Index: [-1, -1, 1]
constraints[minIndex][0]:  [0.0, 1.0, 1.25, 0.0, 0.0, 0.25]
constraints[minIndex][1]:  2.5
results:  [0.5, 10.0, 2.5]
maxIndex:  0
minRatio:  0.25
minIndex:  0
Result Index: [0, -1, 1]
constraints[minIndex][0]:  [1.0, 0.0, -1.875, 0.5, 0.0, -0.375]
constraints[minIndex][1]:  0.25
results:  [0.25, 9.25, 2.5]
maxIndex:  2
minRatio:  1.8048780487804879
minIndex:  1
Result Index: [0, 2, 1]
constraints[minIndex][0]:  [0.0, 0.0, 1.0, -0.2926829268292683, 0.1951219512195122, 0.12195121951219512]


In [76]:
import re

def parse_input(input_string: str) -> tuple:
    input_string = input_string.replace(" ", "")
    terms = re.findall(r'([+-]?\d*\.?\d*)([a-zA-Z]+)', input_string)
    coefficients = []
    variables = []
    for term in terms:
        coefficient = term[0]
        variable = term[1]
        if coefficient in ['+', '-'] or coefficient == '':
            coefficient += '1'
        coefficients.append(float(coefficient))
        variables.append(variable)
    result = input_string.split('=')[-1]
    return [coefficients, float(result)]
equation = "2x1 + 3x2 - 4x3 = 10"
coefficients = parse_input(equation)
print(coefficients)

[[2.0, 3.0, -4.0], 10.0]
